## 외부데이터 불러오기

In [2]:
import pandas as pd
import numpy as np

In [59]:
df_categories = ["1903","1904","1905","1906","1907","1908","1909","1910","1911","1912","2006"]
dfs = {}
df_list = []

for df in df_categories:
    df_name = df
    filename = "../../data/외부/{}.csv".format(df)
    
    dfs[df_name] = pd.read_csv(filename, index_col=0).reset_index()

In [60]:
dfs["1901"] = pd.read_csv("../../data/외부/1912.csv")
dfs["1902"] = pd.read_csv("../../data/외부/2001.csv")

In [63]:
for df in dfs:
    dfs[df]["일"] = dfs[df]["일자"].apply(lambda x:int(str(x)[6:8]))
    dfs[df]["월"] = dfs[df]["일자"].apply(lambda x:int(str(x)[4:6]))
    dfs[df].drop(columns=["일자"], inplace=True)

In [64]:
dfs["1902"]["월"] = dfs["1902"]["월"].replace(1,2)
dfs["1901"]["월"] = dfs["1901"]["월"].replace(12,1)

In [65]:
new_df = None
for df in dfs:
    new_df = pd.concat([new_df, dfs[df]])

In [68]:
new_df.rename(columns={"시간(1시간단위)":"시간"}, inplace=True)

In [69]:
new_df2= new_df.groupby(["월","일","시간","성별","연령대(10세단위)"])["유동인구수"].sum().reset_index()

In [72]:
all_flt = new_df2.groupby(["월","일","시간"])["유동인구수"].sum().reset_index().rename(columns={"유동인구수":"전체유동인구수"})
mean_flt = new_df2.groupby(["월","일","시간"])["유동인구수"].mean().reset_index().rename(columns={"유동인구수":"평균유동인구수"})
sd_flt = new_df2.groupby(["월","일","시간"])["유동인구수"].std().reset_index().rename(columns={"유동인구수":"유동인구수표준편차"})

In [73]:
new_df3 = new_df2.pivot_table(columns=["성별","연령대(10세단위)"], values=["유동인구수"], index=["월","일","시간"]).reset_index()

## 기존 데이터와 결합

In [74]:
df = pd.read_pickle("../../data/pre/merged.pkl")
original_df = df.copy()

In [76]:
new_df3.head()

월  일 시간   유동인구수                                                  \
성별                       남성                                              여성   
연령대(10세단위)               20      30      40      50      60      70      20   
0           1  1  0  755940  825150  801650  745560  474750  318970  733380   
1           1  1  1  753120  822170  799050  743250  473140  317770  730710   
2           1  1  2  749900  819130  796690  741340  471620  316080  728950   
3           1  1  3  745000  816560  794490  739200  469380  313880  727370   
4           1  1  4  742860  813770  791960  736930  467190  312090  725910   

                                                    
성별                                                  
연령대(10세단위)      30      40      50      60      70  
0           783070  805200  790280  523200  440390  
1           781020  804020  789520  522690  439570  
2           779370  802750  788410  521690  438180  
3           777690  801330  786930  520070  436340  
4           776230  799380  785300  518470  434950

In [78]:
original_df["연"] = original_df["방송일시"].dt.year
original_df["월"] = original_df["방송일시"].dt.month
original_df["일"] = original_df["방송일시"].dt.day
original_df["시간"] = original_df["방송일시"].dt.hour
original_df["분"] = original_df["방송일시"].dt.minute
original_df["요일"] = original_df["방송일시"].dt.weekday

In [79]:
tmp_df = pd.merge(original_df, new_df3, on=["월","일","시간"], how="left")

C:\Users\cktna\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,3 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\cktna\anaconda3\lib\site-packages\pandas\core\generic.py:3884: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [105]:
tmp_df2 = pd.merge(tmp_df, all_flt, on=["월","일","시간"], how="left")
tmp_df3 = pd.merge(tmp_df2, mean_flt, on=["월","일","시간"], how="left")
final_df = pd.merge(tmp_df3, sd_flt, on=["월","일","시간"], how="left")

In [106]:
final_df.fillna(method='ffill', inplace=True)

In [108]:
final_df.to_csv("../../data/pre/final1.csv", index=False)